In [0]:
%pip install "albumentations" "numpy<2.0"
dbutils.library.restartPython()

In [0]:
import numpy as np
import pandas as pd
import io
from PIL import Image

import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, BinaryType

import albumentations as A

In [0]:
mount_path = "/mnt/datamount"
dbutils.fs.ls(mount_path)

In [0]:
# 1. Augmentation 파이프라인 정의
TARGET_RESIZE = 256

def custom_solarize(image, threshold=128, **kwargs):
    """Solarize: 임계값보다 밝은 픽셀만 반전"""
    # np.where(조건, True일 때 값, False일 때 값)
    img_out = np.where(image > threshold, 255 - image, image)
    return img_out.astype(np.uint8)

def custom_invert_img(image, **kwargs):
    """InvertImg: 전체 픽셀 반전"""
    return (255 - image).astype(np.uint8)


# 전략 1: '정상' 데이터를 위한 약한 변형 조합
normal_transform = A.Compose([
    A.Resize(TARGET_RESIZE, TARGET_RESIZE),
    A.Rotate(limit=10, p=0.7), # -10~+10도 회전
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.7), # 10% 내외 이동/확대
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8), # 밝기/대비 조절
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=20, val_shift_limit=15, p=0.5),
    # A.GaussianBlur(blur_limit=(3, 7), p=0.5), # 약한 블러
    # A.MotionBlur(blur_limit=(3, 9), p=0.5) 
])

# 전략 2A: 기존 '비정상' 데이터를 위한 기하학적 변형
anomaly_transform_A = A.Compose([
    A.Resize(TARGET_RESIZE, TARGET_RESIZE),
    A.Rotate(limit=15, p=0.8),
    A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=0, p=0.8),
])

# 전략 2B: '정상' 데이터를 '새로운 비정상'으로 만들기 위한 강한 변형
anomaly_transform_B = A.Compose([
    A.Resize(TARGET_RESIZE, TARGET_RESIZE),
    
    # 1. '오점' 생성 (OneOf로 하나 이상 반드시 적용)
    A.OneOf([
        # '검은 점/벗겨짐' (패치형)
        A.CoarseDropout(
            max_holes=8, max_height=32, max_width=32,
            min_holes=1, min_height=16, min_width=16,
            fill_value=0, p=0.8
        ),
        # '크랙(crack)'이나 '긴 스크래치'
        A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1, p=0.7),
        
        # '얼룩'이나 '센서 노이즈'
        A.GaussNoise(var_limit=(100.0, 200.0), p=0.7),
        
        # '압축 깨짐'
        A.ImageCompression(quality_lower=40, quality_upper=70, p=0.5),

        # --- Salt & Pepper ---
        # 'Pepper' (검은색 점) 시뮬레이션
        A.PixelDropout(dropout_prob=0.03, p=0.7, fill_value=0),
        
        # 'Salt' (흰색 점) 시뮬레이션
        A.PixelDropout(dropout_prob=0.03, p=0.7, fill_value=255)

    ], p=1.0),

    # 2. '방해 요소' 추가 (선택적)
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.MotionBlur(blur_limit=(3, 9), p=0.5)
    ], p=0.5),
    
    # 3. 기하학적 변형
    A.Rotate(limit=10, p=0.5)
])



# 2. Augmentation 수량 정의 (목표: 정상 ~800, 비정상 ~200)
# (800 - 18) / 18 = 43.44... -> 44로 올림
N_PER_NORMAL = 44 # 원본 정상(18) -> 18(원본) + 18*44(증강) = 810 (정상)

# 원본 비정상 2개 -> ~50개 (Track A)
# (50 - 2) / 2 = 24 -> 25로 올림 (원본 포함 50개 이상)
N_PER_ANOMALY_A = 25 # 원본 비정상(2) -> 2(원본) + 2*25(증강) = 52 (비정상 Track A)

# 원본 정상 18개 -> ~150개 (Track B)
# 150 / 18 = 8.33... -> 9로 올림
N_PER_NEW_ANOMALY = 9 # 원본 정상(18) -> 18*9(증강) = 162 (비정상 Track B)

# 총 비정상: 52 (Track A) + 162 (Track B) = 214개

In [0]:
# 3. 이미지 변환 헬퍼 함수 (PIL & Numpy)

def bin_to_np(img_bin):
    """Spark BinaryType을 Numpy 배열(RGB)로 변환"""
    try:
        # PIL을 사용해 바이너리를 열고 RGB로 변환
        image = Image.open(io.BytesIO(img_bin)).convert('RGB')
        # Numpy 배열로 변환
        return np.array(image)
    except Exception as e:
        print(f"Error decoding image: {e}")
        return None

def np_to_bin(img_np):
    """
    Numpy 배열(RGB)을 Spark BinaryType(JPEG)으로 변환
    """
    try:
        # Albumentations는 RGB numpy 배열을 반환
        # Image.fromarray는 RGB 순서를 그대로 받음
        image = Image.fromarray(img_np)
        
        # 메모리 내의 바이트 버퍼 생성
        byte_io = io.BytesIO()
        
        # 이미지를 PNG 포맷으로 바이트 버퍼에 저장
        # 원본 특성 유지를 위해 무손실 압축(PNG).
        image.save(byte_io, format='PNG')
        
        # 버퍼의 전체 바이트 값을 반환
        return byte_io.getvalue()
    except Exception as e:
        print(f"Failed to encode image with PIL: {e}")
        return None


# 4. Pandas UDF (Generator) 정의

# UDF가 반환할 스키마 정의
output_schema = StructType([
    StructField("name", StringType(), True),
    StructField("path", StringType(), True),
    StructField("label", StringType(), False),
    StructField("image", BinaryType(), False)
])

# UDF가 반환할 DataFrame의 컬럼 순서 (스키마와 일치해야 함)
SCHEMA_COLUMNS = ["name", "path", "label", "image"]

# UDF 1: 정상 이미지 처리용
def augment_normal_generator(pdf_iter):
    for pdf in pdf_iter:
        # 이 파티션에서 생성될 모든 행을 저장할 리스트
        output_rows = []

        for _, row in pdf.iterrows():
            name, path, label = row['name'], row['path'], row['label']
            
            img_np = bin_to_np(row['image'])
            if img_np is None:
                continue

            # 1. 원본 이미지 추가
            output_rows.append((name, path, label, row['image']))

            # 2. '정상' 증강 (Strategy 1)
            for i in range(N_PER_NORMAL):
                augmented_np = normal_transform(image=img_np)['image']
                aug_bin = np_to_bin(augmented_np)
                if aug_bin:
                    new_name = f"{name}_aug_normal_{i}"
                    output_rows.append((new_name, "N/A", label, aug_bin))

            # 3. '새로운 비정상' 생성 (Strategy 2B)
            for i in range(N_PER_NEW_ANOMALY):
                augmented_np = anomaly_transform_B(image=img_np)['image']
                aug_bin = np_to_bin(augmented_np)
                if aug_bin:
                    new_name = f"{name}_aug_new_anomaly_{i}"
                    output_rows.append((new_name, "N/A", "noise", aug_bin))
        # 파티션의 모든 행 처리가 끝나면, 리스트를 DataFrame으로 변환하여 yield
        if output_rows:
            yield pd.DataFrame(output_rows, columns=SCHEMA_COLUMNS)

# UDF 2: 비정상 이미지 처리용
def augment_anomaly_generator(pdf_iter):
    for pdf in pdf_iter:
        # 이 파티션에서 생성될 모든 행을 저장할 리스트
        output_rows = []

        for _, row in pdf.iterrows():
            name, path, label = row['name'], row['path'], row['label']
            
            img_np = bin_to_np(row['image'])
            if img_np is None:
                continue

            # 1. 원본 비정상 이미지 반환
            output_rows.append((name, path, label, row['image']))

            # 2. '비정상' 증강 (Strategy 2A)
            for i in range(N_PER_ANOMALY_A):
                augmented_np = anomaly_transform_A(image=img_np)['image']
                aug_bin = np_to_bin(augmented_np)
                if aug_bin:
                    new_name = f"{name}_aug_anomaly_{i}"
                    output_rows.append((new_name, "N/A", label, aug_bin))
        # 파티션의 모든 행 처리가 끝나면, 리스트를 DataFrame으로 변환하여 yield
        if output_rows:
            yield pd.DataFrame(output_rows, columns=SCHEMA_COLUMNS)



In [0]:
# 5. Spark 작업 실행
print("원본 DataFrame 로드 중...")
df = spark.read.format('parquet').load(f'{mount_path}/images_final')


# 데이터 분리 및 캐시
df_normal = df.filter(F.col('label') == 'surfing').cache()
df_anomaly = df.filter(F.col('label') == 'noise').cache()

print(f"정상 원본: {df_normal.count()}, 비정상 원본: {df_anomaly.count()}")

print("Augmentation 시작 (UDF 1: Normal -> Normal + New Anomaly)...")
df_aug_1 = df_normal.mapInPandas(augment_normal_generator, schema=output_schema)

print("Augmentation 시작 (UDF 2: Anomaly -> Anomaly)...")
df_aug_2 = df_anomaly.mapInPandas(augment_anomaly_generator, schema=output_schema)

# 두 결과 합치기
df_augmented = df_aug_1.unionByName(df_aug_2)

# 결과 확인
print("Augmentation 완료. 최종 결과 집계:")
df_augmented.groupBy('label').count().show()

# (선택 사항) 캐시 해제
df_normal.unpersist()
df_anomaly.unpersist()

In [0]:
# define image metadata
img_meta = {
    "spark.contentAnnotation":'{"mimeType":"image/jpeg"}'
}

df_augmented = df_augmented.withColumn('image', F.col('image').alias('image', metadata = img_meta))
display(df_augmented, limit = 10)

In [0]:
df_augmented.write.mode("overwrite").format('parquet').save(f'{mount_path}/images_augmented')